<a href="https://colab.research.google.com/github/hhandc/facerec_password/blob/main/Password_encrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment


In [ ]:
!pip install face_recognition
!pip install opencv-python
!sudo apt-get install cmake
!pip install dlib
!pip install cryptography
!pip install dlib --no-binary :all:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=be240df32c4b14dfb3e87812ee99366c4ac4de78bb033cb4e4d8142d410bbe69
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


# Preprocess images quality

In [ ]:
import cv2
import numpy as np
import os

def process_image(image_path, output_path):
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error reading image: {image_path}")
        return

    denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

    lab = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged_lab = cv2.merge((cl, a, b))
    contrast_image = cv2.cvtColor(merged_lab, cv2.COLOR_LAB2BGR)

    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened_image = cv2.filter2D(contrast_image, -1, kernel)

    cv2.imwrite(output_path, sharpened_image)

def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Processing {filename}...")
            process_image(input_path, output_path)
        else:
            print(f"Skipping non-image file: {filename}")

if __name__ == "__main__":
    input_folder = "input_images"
    output_folder = "output_images"

    process_folder(input_folder, output_folder)
    print("Processing complete!")

In [ ]:
import face_recognition
import cv2

def extract_face_features(image_path):
    image = face_recognition.load_image_file(image_path)
    face_landmarks_list = face_recognition.face_landmarks(image)

    if not face_landmarks_list:
        return "No faces detected in the image."

    for face_landmarks in face_landmarks_list:
        face_features_str = "Facial Features:\n"
        for feature, points in face_landmarks.items():
            face_features_str += f"{feature}: {points}\n"
        return face_features_str

image_path = input("Enter the path to the image file: ")
features = extract_face_features("selfie.jpg")
print(features)

KeyboardInterrupt: Interrupted by user

In [ ]:
import json

website_random_strings = {
    "example.com": "a8K3Ld9X",
    "mywebsite.org": "b9P4Xm7N",
    "anotherwebsite.net": "p5C7Rt8V",
    "somedomain.com": "q1Z8Mn4S"
}

with open("websites.json", "w") as f:
    json.dump(website_random_strings, f, indent=4)

In [ ]:
import face_recognition
import numpy as np
import hashlib
import random
import string
from cryptography.fernet import Fernet
import base64
import json
import os

def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def extract_face_distances(image_path):
    image = face_recognition.load_image_file(image_path)
    face_landmarks_list = face_recognition.face_landmarks(image)

    if not face_landmarks_list:
        return None, None, "No faces detected in the image."

    face_landmarks = face_landmarks_list[0]

    left_eye = face_landmarks['left_eye']
    right_eye = face_landmarks['right_eye']

    left_eye_center = np.mean(left_eye, axis=0)
    right_eye_center = np.mean(right_eye, axis=0)
    eye_distance = calculate_distance(left_eye_center, right_eye_center)

    chin = face_landmarks['chin']
    nose_bridge = face_landmarks['nose_bridge']

    forehead_point = nose_bridge[0]
    chin_point = chin[-1]

    forehead_to_chin_distance = calculate_distance(forehead_point, chin_point)

    return eye_distance, forehead_to_chin_distance, None

def generate_random_string(length=8):
    letters_and_digits = string.ascii_letters + string.digits
    return ''.join(random.choice(letters_and_digits) for i in range(length))

def get_or_create_random_string(website, storage_file="websites.json"):
    if not os.path.exists(storage_file):
        with open(storage_file, "w") as f:
            json.dump({}, f)

    with open(storage_file, "r") as f:
        website_random_strings = json.load(f)

    if website in website_random_strings:
        print(website_random_strings[website])
        return website_random_strings[website]
    else:
        random_str = generate_random_string(8)
        website_random_strings[website] = random_str

        with open(storage_file, "w") as f:
            json.dump(website_random_strings, f)

        return random_str

def generate_encryption_key(image_path, website):
    eye_distance, forehead_to_chin_distance, error = extract_face_distances(image_path)

    if error:
        return None, error

    eye_distance_str = f"{int(eye_distance)}"
    forehead_to_chin_distance_str = f"{int(forehead_to_chin_distance)}"

    combined_str = eye_distance_str + forehead_to_chin_distance_str

    random_str = get_or_create_random_string(website)
    combined_with_random = combined_str + random_str

    hashed_key = hashlib.sha256(combined_with_random.encode()).digest()
    encryption_key = base64.urlsafe_b64encode(hashed_key[:32])

    return encryption_key, None

def encrypt_password(password, encryption_key):
    fernet = Fernet(encryption_key)
    encrypted_password = fernet.encrypt(password.encode())

    return encrypted_password

def decrypt_password(encrypted_password, encryption_key):
    fernet = Fernet(encryption_key)
    decrypted_password = fernet.decrypt(encrypted_password).decode()

    return decrypted_password

image_path = input("Enter the path to the image file: ")
website = input("Enter the website you want to use this password: ")
encryption_key, error = generate_encryption_key(image_path, website)

if encryption_key:
    print("Encryption Key Generated Successfully!")
    password = input("Enter the password you want to encrypt: ")
    encrypted_password = encrypt_password(password, encryption_key)
    print("Encrypted Password:", encrypted_password)

    decrypted_password = decrypt_password(encrypted_password, encryption_key)
    print("Decrypted Password:", decrypted_password)
else:
    print(f"Error: {error}")

KeyboardInterrupt: Interrupted by user

In [ ]:
import face_recognition
import numpy as np
import hashlib
import random
import string
import json
import os
from cryptography.fernet import Fernet
import base64

def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def extract_face_distances(image_path):
    image = face_recognition.load_image_file(image_path)
    face_landmarks_list = face_recognition.face_landmarks(image)

    if not face_landmarks_list:
        return None, None, "No faces detected in the image."

    face_landmarks = face_landmarks_list[0]
    left_eye = face_landmarks['left_eye']
    right_eye = face_landmarks['right_eye']
    left_eye_center = np.mean(left_eye, axis=0)
    right_eye_center = np.mean(right_eye, axis=0)
    eye_distance = calculate_distance(left_eye_center, right_eye_center)

    chin = face_landmarks['chin']
    nose_bridge = face_landmarks['nose_bridge']
    forehead_point = nose_bridge[0]
    chin_point = chin[-1]
    forehead_to_chin_distance = calculate_distance(forehead_point, chin_point)

    return eye_distance, forehead_to_chin_distance, None

def get_or_create_random_string(website, storage_file="website_random_strings.json"):
    if not os.path.exists(storage_file):
        with open(storage_file, "w") as f:
            json.dump({}, f)

    with open(storage_file, "r") as f:
        website_random_strings = json.load(f)

    if website in website_random_strings:
        return website_random_strings[website]
    else:
        random_str = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(8))
        website_random_strings[website] = random_str
        with open(storage_file, "w") as f:
            json.dump(website_random_strings, f)
        return random_str

def generate_encryption_key(image_path, website):
    eye_distance, forehead_to_chin_distance, error = extract_face_distances(image_path)
    if error:
        return None, error

    eye_distance_str = f"{eye_distance:.2f}"
    forehead_to_chin_distance_str = f"{forehead_to_chin_distance:.2f}"
    combined_str = eye_distance_str + forehead_to_chin_distance_str
    random_str = get_or_create_random_string(website)
    combined_with_random = combined_str + random_str
    print(combined_with_random)
    hashed_key = hashlib.sha256(combined_with_random.encode()).digest()
    encryption_key = base64.urlsafe_b64encode(hashed_key[:32])

    return encryption_key, None

def encrypt_password(password, encryption_key):
    fernet = Fernet(encryption_key)
    encrypted_password = fernet.encrypt(password.encode())
    return encrypted_password

def calculate_consistency(image_folder, website, password):
    encrypted_passwords = []
    consistent_count = 0
    total_images = 0

    for filename in os.listdir(image_folder):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(image_folder, filename)
            encryption_key, error = generate_encryption_key(image_path, website)
            if encryption_key:
                encrypted_password = encrypt_password(password, encryption_key)
                print(encrypted_password)
                encrypted_passwords.append(encrypted_password)
                total_images += 1

    if total_images > 0:
        first_password = encrypted_passwords[0]
        consistent_count = sum(1 for ep in encrypted_passwords if ep == first_password)
        consistency_rate = (consistent_count / total_images) * 100
        accuracy = "Consistent" if consistent_count == total_images else "Inconsistent"

        print(f"Consistency Rate: {consistency_rate:.2f}%")
        print(f"Accuracy: {accuracy}")
    else:
        print("No valid images found for processing.")

# Example usage
image_folder = input("Enter the path to the folder of images: ")
website = input("Enter the website URL: ")
password = input("Enter the password you want to encrypt: ")

calculate_consistency(image_folder, website, password)

Enter the path to the folder of images: image5
Enter the website URL: example.com
Enter the password you want to encrypt: 1234
554.23690.160hIiyIJy
b'gAAAAABnJ6LdhE9qOk8tiDiy4EZitbwdQYNEuRI420hxwqnlDo6mLIFOT-aKplDgrZIP7Jr5QvR18w7c7gAzRutsoYdnUro4ag=='
554.23690.160hIiyIJy
b'gAAAAABnJ6Lhvfdzug_P5pj-EUMIxS9Mjbn9zPCxS7VMED4JyolPjKxppAUR-LZ1wLO0HsIeRYm2CpRz23aUGiX5mJOvjTvC_A=='
Consistency Rate: 50.00%
Accuracy: Inconsistent


In [ ]:
import face_recognition
import numpy as np
import hashlib
import random
import string
import json
import os
from cryptography.fernet import Fernet
import base64

def get_or_create_random_string(website, storage_file="website_random_strings.json"):
    if not os.path.exists(storage_file):
        with open(storage_file, "w") as f:
            json.dump({}, f)

    with open(storage_file, "r") as f:
        website_random_strings = json.load(f)

    if website in website_random_strings:
        return website_random_strings[website]
    else:
        random_str = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(8))
        website_random_strings[website] = random_str
        with open(storage_file, "w") as f:
            json.dump(website_random_strings, f)
        return random_str

def generate_encryption_key_from_embedding(image_path, website):
    image = face_recognition.load_image_file(image_path)
    embeddings = face_recognition.face_encodings(image)

    if not embeddings:
        return None, "No face detected."

    embedding = embeddings[0]

    embedding_str = ''.join(f"{value:.5f}" for value in embedding)

    random_str = get_or_create_random_string(website)
    combined_str = embedding_str + random_str

    hashed_key = hashlib.sha256(combined_str.encode()).digest()[:32]
    encryption_key = base64.urlsafe_b64encode(hashed_key).decode('utf-8')
    encryption_key = encryption_key[:32].ljust(32, '=')

    return encryption_key.encode(), None

def encrypt_password(password, encryption_key):
    fernet = Fernet(encryption_key)
    encrypted_password = fernet.encrypt(password.encode())
    return encrypted_password

def decrypt_password(encrypted_password, encryption_key):
    fernet = Fernet(encryption_key)
    decrypted_password = fernet.decrypt(encrypted_password).decode()
    return decrypted_password

def calculate_consistency(image_folder, website, password):
    encrypted_passwords = []
    total_images = 0

    for filename in os.listdir(image_folder):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(image_folder, filename)
            encryption_key, error = generate_encryption_key_from_embedding(image_path, website)

            if error:
                print(f"Error for {filename}: {error}")
                continue

            encrypted_password = encrypt_password(password, encryption_key)
            encrypted_passwords.append(encrypted_password)
            total_images += 1

    if total_images > 0:
        first_password = encrypted_passwords[0]
        consistent_count = sum(1 for ep in encrypted_passwords if ep == first_password)
        consistency_rate = (consistent_count / total_images) * 100
        accuracy = "Consistent" if consistent_count == total_images else "Inconsistent"

        print(f"Consistency Rate: {consistency_rate:.2f}%")
        print(f"Accuracy: {accuracy}")
    else:
        print("No valid images found for processing.")

image_folder = input("Enter the path to the folder of images: ")
website = input("Enter the website URL: ")
password = input("Enter the password you want to encrypt: ")

calculate_consistency(image_folder, website, password)

Enter the path to the folder of images: image5
Enter the website URL: example.com
Enter the password you want to encrypt: 1234


ValueError: Fernet key must be 32 url-safe base64-encoded bytes.